Draft Recognizer

This app takes an image link and classifies the image as 'draft' or 'not_draft'. There is also a button to display the most recently downloaded image that is being classified.

In [16]:
%matplotlib inline

# Credit to joedockrill's code that I used to debug a lot of things
# https://github.com/joedockrill/heroku
# including downloading a different torch file that reduced the size from
# about 700+mb to ~150mb

# To do:
# Figure out a way to let the code handle jpg files
# Figure out a way to allow direct uploads of jpg or png files

from IPython.display import clear_output, Image

from matplotlib.pyplot import imshow, axis, show
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import re
import os

from fastai.vision import *
import urllib.request

import warnings

# Ignore warnings for webapp
warnings.filterwarnings('ignore')
# Wasn't able to find a way to just ignore warnings from a specific
# function; ignoring the warnings from clicking classify didn't work
# because it still popped up

def gdrive_share_to_id(link):
  # Function that takes the Google Drive share link and spits out the
  # the export/download link
  link2 = re.sub('^.*file\/d\/', '', link)
  link2 = re.sub('\/view\?usp=sharing$', '', link2)
  return(link2)

id1 = gdrive_share_to_id("https://drive.google.com/file/d/1u7sONyh8MhbEF7nrvgtuIDpU_BOreNwX/view?usp=sharing")


def download_file_from_google_drive(id, destination):
  # download_file_from_google_drive code taken from tttthomasssss
  # https://stackoverflow.com/a/3922503
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)

download_file_from_google_drive(id1, './export.pkl')

defaults.device = torch.device('cpu')
learn = load_learner(".")

# Notes
# This webapp may be vulnerable to large files that are downloaded from the URLs
# or from direct file uploads. I'm not sure how to handle this.

# Widget for receiving an image link
link1 = widgets.Text(
    description = "Image link: ",
    placeholder = "Type image link here"
)

# Widget that downloads the image when clicked
download1 = widgets.Button(
    description = 'Upload from link',
    disabled = False,
    button_style = '', # 'success', 'info', 'warning', 'danger' or ''
    tooltip = 'Machine downloads the image supplied by the link',
)

def ext_to_fname(ext):
  # Function that takes the image extension of a URl
  # and returns the filename with the appropriate extension
	switcher = { 
		"png": "image.png", 
		"jpg": "image.jpg", 
		"jpeg": "image.jpeg"
	}
	return switcher.get(ext, "Error") 

def link_ext(imglink):
  # Function that takes an image link and returns .png, .jpg, or .jpeg
  ext = re.sub('.*\.(?=(((png)|(jpg))|(jpeg))$)', '', imglink)
  return(ext)

def dl_img(event):
  with d1_out:
    d1_out.clear_output()
    # Save the image from the link to 'image.png'
    bla = link1
    # Check that input is a string
    if(type(bla.value) is not str):
      print("Error, link must be a string!")
    # Check that input link leads to an image
    elif(re.search('\.(((png)|(jpg))|(jpeg))$', bla.value) is None):
      print('URL must link to a .png, .jpg, or .jpeg file!')
    else:
      try: # Check that there is an image at the link
        ext1 = link_ext(bla.value)
        urllib.request.urlretrieve(bla.value, ext_to_fname(ext1))
        print('Image downloaded to ' + ext_to_fname(ext1) + '!')
        # Check that the file can be opened
        path1 = './' + ext_to_fname(ext1)
        file = open(path1, 'rb')
      except: # If the image in the link is not found
        print('image not found :( will classify "default.png" instead')
download1.on_click(dl_img)

# Widget that classifies the image when clicked
classify1 = widgets.Button(
    description = 'Classify!',
    disabled = False,
    button_style = '', # 'success', 'info', 'warning', 'danger' or ''
    tooltip = 'Classify the image as "draft" or "not draft"',
)

def classify_img(event):
  with c1_out:
    c1_out.clear_output()
    try:
      path1 = './' + ext_to_fname(link_ext(link1.value))
      img = open_image(Path(path1))
    except:
      img = open_image(Path('./default.png'))
      print("Image not found, default loaded instead")
    pred_class,pred_idx,outputs = learn.predict(img)
    print('Class: ' + pred_class.obj)
classify1.on_click(classify_img)

# Widget that shows the downloaded image
img1 = widgets.Button(
    description = 'Refresh image',
    disabled = False,
    button_style = '', # 'success', 'info', 'warning', 'danger' or ''
    tooltip = 'View the image',
)

def view_img(event):
  with img_out:
    img_out.clear_output(True)
    try:
      # Get link1
      fname1 = ext_to_fname(link_ext(link1.value))
      # Check that the file can be opened
      path1 = './' + fname1
      # put link one into imread
      img = mpimg.imread(path1)
    except:
      print("Image not found, default loaded instead")
      img = mpimg.imread('default.png')
    imgplot = plt.imshow(img)
    plt.axis('off')
    plt.show()
img1.on_click(view_img)

## Output widgets
# Show the link that's being received
def f1(link):
  print(' Link received: {}'.format(link))
out = widgets.interactive_output(f1, {'link': link1})

# Print statement that confirms the download
d1_out = widgets.Output(layout = Layout(margin='auto'))

# Print statement that shows the classification
c1_out = widgets.Output(layout = Layout(margin='auto'))

# Print image
img_out = widgets.Output(layout = Layout(margin = 'auto'))

# Layout of all the widgets
widgets.VBox([
              widgets.HBox([widgets.VBox([link1, download1, classify1]),
              widgets.VBox([out, d1_out, c1_out])]),
              widgets.HBox([img1, img_out])
])